This is the final version of the training workflow/pipeline

In [1]:
# import the necessary libraries
from ultralytics import YOLO
import os
import shutil
from megadetector.detection import run_detector
import cv2
import pandas as pd
from PIL import Image
from matplotlib import pyplot as plt
import re
%matplotlib inline

In [3]:
# megadetector = run_detector.load_detector('MDV5A')
def crop_image_normalized(img, normalized_coords):
    try:
        
        height, width, _ = img.shape
        x_min = int(normalized_coords[0] * width)
        y_min = int(normalized_coords[1] * height)
        box_width = int(normalized_coords[2] * width)
        box_height = int(normalized_coords[3] * height)
        # Crop the image
        cropped_img = image[y_min:y_min + box_height, x_min:x_min + box_width]
        return cropped_img

    except FileNotFoundError:
        print(f"Error: Image not found at {image_path}")
        return None
    except ValueError:
        print(f"Error: Invalid normalized coordinates format: {normalized_coords}. Expected 'x y w h'.")
        return None

Bypassing download of already-downloaded file md_v5a.0.0.pt
Model v5a.0.0 available at C:\Users\willo\AppData\Local\Temp\megadetector_models\md_v5a.0.0.pt
Loading PT detector with compatibility mode classic


Fusing layers... 
Fusing layers... 
Model summary: 733 layers, 140054656 parameters, 0 gradients, 208.8 GFLOPs
Model summary: 733 layers, 140054656 parameters, 0 gradients, 208.8 GFLOPs


In [ ]:
all_detections = []
image_source_directory = "E:\\Wild deserts photos\\2024_11_WCAM_originals\\WCAM15"
image_training_output = f"{image_source_directory}\\training"
image_paths = []
for dirpath, dirnames, filenames in os.walk(image_source_directory):
    for filename in filenames:
        if filename.endswith(".JPG"):
            image_path = os.path.join(dirpath, filename)
            image_paths.append(image_path)
for image_path in image_paths:
# try:
    print(f"Processing image: {image_path}")
    # get time of day from image
    image = cv2.imread(image_path)
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    hue = hsv_image[:, :, 0].mean()  # H channel represents hue
    # Adjust the thresholds as needed
    if hue > 10:
        model_required = 'day'
    else:
        model_required = 'night'

    print(f"Image is a {model_required} image")

    if model_required == 'night':
        model = YOLO("E:\\Wild deserts photos\\model_training\\runs\\classify\\classificationv1_night\\weights\\best.pt")
    else:
        model = YOLO("E:\\Wild deserts photos\\model_training\\runs\\classify\\classificationv1_day\\weights\\best.pt")


    camera = re.search(r'WCAM(\d+)', image_path)[0]
    detections = megadetector.generate_detections_one_image(image)
    datetime = Image.open(image_path)._getexif()[36867].replace(' ', '_').replace(':', '_')
    above = [d for d in detections['detections'] if (d['conf'] > 0.7) and (d['category'] == '1')]
    for idx, detection in enumerate(above):
        bbox_normalized = detection['bbox']
        cropped_img = crop_image_normalized(image, bbox_normalized)
        
        result = model.predict(
            cropped_img, 
            save=False, # I don't save anything here because I want to sort into folders
            save_txt=False,
            save_conf=False)[0]
        species = result.names[result.probs.top1]
        conf = int(result.probs.top1conf)
        output_dir = f"{image_training_output}\\{model_required}\\{species}"
        print(f"Saving to {output_dir}")
        os.makedirs(output_dir, exist_ok=True)
        output_filename = f"{output_dir}\\{camera}_{datetime}_{species}_{idx}.jpg"
        cv2.imwrite(output_filename, cropped_img)
        df = pd.DataFrame({
                    'original_path': [image_path],
                    'cropped_path': [output_filename],
                    'species': [species],
                    'confidence': [conf],
                    'bbox': [bbox_normalized],
                    'time': [datetime],
                    'model': [model_required],
                    'camera': [camera] 
                })
        all_detections.append(df)
# except Exception as e:
#     print(f"Error processing {image_path}: {e}")
if all_detections:
    final_df = pd.concat(all_detections, ignore_index=True)
    final_df.to_csv(f"{image_source_directory}\\output_{camera}.csv", index=False)


Processing image: E:\Wild deserts photos\2024_11_WCAM_originals\WCAM16\RCNX0054.JPG
Image is a day image

0: 224x224 Kangaroo 1.00, Dingo 0.00, Rabbit 0.00, Cat 0.00, 152.5ms
Speed: 7.0ms preprocess, 152.5ms inference, 0.0ms postprocess per image at shape (1, 3, 224, 224)
Saving to E:\Wild deserts photos\2024_11_WCAM_originals\WCAM16\training\day\Kangaroo

0: 224x224 Kangaroo 1.00, Dingo 0.00, Rabbit 0.00, Cat 0.00, 154.1ms
Speed: 12.0ms preprocess, 154.1ms inference, 0.0ms postprocess per image at shape (1, 3, 224, 224)
Saving to E:\Wild deserts photos\2024_11_WCAM_originals\WCAM16\training\day\Kangaroo
Processing image: E:\Wild deserts photos\2024_11_WCAM_originals\WCAM16\RCNX0050.JPG
Image is a day image

0: 224x224 Kangaroo 1.00, Rabbit 0.00, Dingo 0.00, Cat 0.00, 196.2ms
Speed: 6.0ms preprocess, 196.2ms inference, 0.0ms postprocess per image at shape (1, 3, 224, 224)
Saving to E:\Wild deserts photos\2024_11_WCAM_originals\WCAM16\training\day\Kangaroo

0: 224x224 Kangaroo 1.00, Din